## Importing the libraries

In [701]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.svm import SVR

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import r2_score

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the dataset

In [702]:
df = pd.read_csv('/content/drive/MyDrive/Biogas RMS project/Datasets/Paper 13/paper13.csv')
df.head()

,Biomass type,Reactor/feeding,VS (%),pH,OLR (g VS/l.d),HRT (d),T (°C),Reactor Volume (m³),Cumulated biogas volume (L/(g VS))
0,0,0,10.00,7.62,0.6270,19.2,55,0.0500,0.0668
1,0,2,15.30,8.00,3.1702,47.0,37,0.0473,0.6765
2,0,0,4.78,7.25,1.2400,15.0,37,0.0450,0.8227
3,0,0,4.78,7.25,1.7600,15.0,37,0.0450,0.6219
4,0,2,6.36,7.30,3.2000,25.0,35,0.0400,0.5755


## Outlier Detection and Removal


## 1. VS%

In [703]:
percentile25 = df['VS (%)'].quantile(0.25)
percentile75 = df['VS (%)'].quantile(0.75)

iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

print(upper_limit)
print(lower_limit)

# Removing outliers
df = df[df['VS (%)'] <= upper_limit]

df.shape

25.784999999999997
-6.7349999999999985


(106, 9)

## 2. OLR

In [704]:
# Using the percentile method of outlier removal

upper_limit = df['OLR (g VS/l.d)'].quantile(0.99)
lower_limit = df['OLR (g VS/l.d)'].quantile(0.01)

print(upper_limit)
print(lower_limit)

# Removing outliers
df = df[df['OLR (g VS/l.d)'] <= upper_limit]

df.shape

16.002800000000008
0.2


(104, 9)

## PCA

### Picking out the numerical values and standard scaling the data

In [705]:
# # Using standard Scaling

X = df.drop(columns=['Cumulated biogas volume (L/(g VS))'])
y = df['Cumulated biogas volume (L/(g VS))']

# Getting numerical values
X_num = X.drop(columns=['Biomass type', 'Reactor/feeding'])
# Scaling the data
scaler = StandardScaler()
scaler.fit(X_num)
X_num_trans = scaler.transform(X_num)

In [706]:
# Using power transforms

# X = df.drop(columns=['Cumulated biogas volume (L/(g VS))'])
# y = df['Cumulated biogas volume (L/(g VS))']

# # Getting numerical values
# X_num = X.drop(columns=['Biomass type', 'Reactor/feeding', 'T (°C)'])
# X_temp = X['T (°C)']

# # Scaling the data
# scaler = StandardScaler()
# X_temp = X_temp.to_numpy().reshape(-1,1)
# X_temp = scaler.fit_transform(X_temp)

# # Power Transform
# powerTrans = PowerTransformer()
# X_num = powerTrans.fit_transform(X_num)

# X_num_trans =  np.hstack(( X_num, X_temp))
# print(X_num_trans.shape)

### Performing the PCA
The number of dimensions that are retained is decided by the amount of variance that is explained by the components.

In [707]:
# PCA that retains only those dimensions that explain at least 95% of vairance
from sklearn.decomposition import PCA


pca = PCA(n_components=6)
pca.fit(X_num_trans)
X_num_reduced = pca.transform(X_num_trans)

print(X_num_trans.shape)
print(X_num_reduced.shape)


(104, 6)
(104, 6)


## One hot encoding the categorical values

In [708]:
X_cat = X[['Biomass type', 'Reactor/feeding']]

ohe = OneHotEncoder(drop="first")
ohe.fit(X_cat)

X_cat_trans = ohe.transform(X_cat).toarray()

X is reconstructed after applying OHE

In [709]:
X_trans = np.hstack(( X_num_reduced,X_cat_trans,))
X_trans.shape

(104, 12)

## Splitting the data into Test and training data

In [710]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, test_size = 0.2, random_state = 3)

# Pipeline 

Since all scaling and ohe has been performed already, this will have only the model training step.



In [711]:
# Training
trf1 = SVR(kernel = 'rbf')

## Creating the pipeline

In [712]:
pipe = Pipeline([
    ('svr', trf1),
])

## Training and predicting using the pipeline

Here we are using default parameters

In [713]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')
# Show the steps involved in the pipeline
pipe.named_steps

# train and predict
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test, y_pred)

# Visualize 
# X_trans = pipe.fit_transform(X_train)
# X_pd = pd.DataFrame(X_trans)
# X_pd.head()

0.8322482535971867

## GridSearch using the pipeline

In [714]:
# gridsearchcv
params = {
    'svr__kernel':['rbf', 'poly', 'sigmoid'],
    'svr__degree': [2, 3],
    'svr__C' : [1, 10, 100],
    'svr__tol' : [1e-3, 1e-2]

}

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=10, scoring='r2')
grid.fit(X_trans, y)


GridSearchCV(cv=10, estimator=Pipeline(steps=[('svr', SVR())]),
             param_grid={'svr__C': [1, 10, 100], 'svr__degree': [2, 3],
                         'svr__kernel': ['rbf', 'poly', 'sigmoid'],
                         'svr__tol': [0.001, 0.01]},
             scoring='r2')

# GridSearchCV results


In [715]:
print("R2 score: ", round(r2_score(y_test, y_pred), 4), "  ")
print(" CV score: ", round(grid.best_score_, 4), "  ")

R2 score:  0.8322   
 CV score:  0.3721   


In [716]:
grid.best_params_

{'svr__C': 1, 'svr__degree': 2, 'svr__kernel': 'rbf', 'svr__tol': 0.001}

# Results: 

## Standard Scaling

1. 6 components  
  R2 score:  0.8322  
  CV score:  0.3721  

2. 5 components  
  R2 score:  0.8086  
  CV score:  0.3864  

3. 4 components  
  R2 score:  0.7087   
  CV score:  0.351   

4. 3 components  
  R2 score:  0.6836   
 CV score:  0.3303 

5. 2 components  
  R2 score:  0.6461   
  CV score:  0.2933   

6. 1 component  
  R2 score:  0.4841   
 CV score:  -0.0363 

## Power Transform

1. 6 components  
  R2 score:  0.8512   
 CV score:  0.4333     

2. 5 components  
  R2 score:  0.8396   
 CV score:  0.4366   

3. 4 components  
  R2 score:  0.7448   
 CV score:  0.3331   

4. 3 components  
  R2 score:  0.7758   
 CV score:  0.3377   

5. 2 components  
  R2 score:  0.6499   
 CV score:  0.1801   

6. 1 component  
  R2 score:  0.4976   
 CV score:  0.033 